# 2º Trabalho Prático: Planejamento de Caminhos

#### Importação de Bibliotecas

In [ ]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import math
import time
import numpy as np
import heapq
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D

#### Funções Compartilhadas

In [ ]:
# Normalize angle to the range [-pi,pi)
def normalize_angle (angle):
    return np.mod (angle + np.pi, 2 * np.pi) - np.pi

def rotation_z (theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta),  np.cos(theta), 0],
                     [            0,              0, 1]])

#### Funções Roadmap

In [ ]:
def read_binary_image (image_address):
    
    image = mpimg.imread (image_address)
    for row in range(len(image)):
        for col in range(len(image[row])):
            image[row][col] = 1 if image[row][col] < 1 else 0
            
    return image

def print_world_grid (world_grid, img_id = ""):
    global num_img
    
    x_len = len (world_grid[0])
    y_len = len (world_grid)
    
    figure = plt.figure (figsize = (40, 40 * y_len / x_len), dpi = 100)
    ax = figure.add_subplot (111, aspect = 'equal')

    ax.grid (which = 'major', axis = 'both', linestyle = '-', color = 'k', linewidth = .5)
    ax.set_xticks (np.arange (0, x_len))
    ax.set_yticks (np.arange (0, y_len))
    
    ax.imshow (1 - world_grid, cmap = 'Blues', extent = (0, x_len, 0, y_len), alpha = 0.6)
    
    #plt.savefig(f"world_{img_id}.png")
    plt.close()
        

In [ ]:
class World:
    
    def __init__ (self, image_address, real_dimensions):
        self.image_address = image_address
        self.world_matrix = read_binary_image (self.image_address)
        self.real_dimensions = real_dimensions
        self.scale = self.world_matrix.shape[:2] / self.real_dimensions
        
    def is_cell_area_free (self, row, col, cell_size):
        area_size = cell_size * self.scale
        left = int (col * area_size[0])
        right = int ((col + 1) * area_size[0])
        top = int (row * area_size[1])
        bottom = int ((row + 1) * area_size[1])
        return 1 if np.sum (self.world_matrix[top:bottom, left:right]) == 0 else 0
        
    def discretize_world (self, cell_size = 1):
        num_rows, num_cols = (self.real_dimensions / cell_size).astype(int)
        grid = np.zeros ((num_rows, num_cols))
        for row in range (num_rows):
            for col in range (num_cols):
                grid[row][col] = self.is_cell_area_free (row, col, cell_size)

        print_world_grid(grid)
        return grid

In [ ]:
#moves the referential from the center of the world to the top left corner of the grid and finds the correspondent cell
def find_grid_position (global_pos, cell_size, grid_size):

    x_pos = np.floor (global_pos[0] / cell_size + grid_size[0] / 2)
    y_pos = np.floor (-global_pos[1] / cell_size + grid_size[1] / 2)

    return np.array ((x_pos, y_pos)).astype(int)

def find_map_position (grid_pos, cell_size, grid_size):

    x_pos = (grid_pos[0] + 0.5 - grid_size[0] / 2) * cell_size
    y_pos = -(grid_pos[1] + 0.5 - grid_size[1] / 2) * cell_size

    return np.array ((x_pos, y_pos))

def find_goal (robot, goal, world_map, cell_size):

    robot_cell = find_grid_position (robot.global_position, cell_size, [len (world_map[0]), len (world_map)])
    goal_cell = find_grid_position (goal.global_position, cell_size, [len (world_map[0]), len (world_map)])
    
    grid_path = A_star (robot_cell, goal_cell, world_map)

    world_path = [find_map_position (stop, cell_size, [len (world_map[0]), len (world_map)]) for stop in grid_path]
    
    robot.follow_path (np.flip (world_path, 0))

In [ ]:
class Node:

    def __init__ (self, grid_position, parent, current_cost):
        self.grid_position = grid_position
        self.parent = parent
        self.current_cost = current_cost
        self.priority_score = current_cost #If I don't call calc_priority_score, the algorithm works like BFS by default

    def __lt__ (self, other):
        return self.priority_score < other.priority_score

    def manhattan_distance (self, goal_pos):
        return abs (self.grid_position[0] - goal_pos[0]) + abs (self.grid_position[1] - goal_pos[1])
        
    def calc_priority_score (self, goal_pos):
        self.heuristic_cost = self.manhattan_distance (goal_pos)
        self.priority_score = self.current_cost + self.heuristic_cost

In [ ]:
client = RemoteAPIClient()
sim = client.require('sim')

class CoppeliaObject:
    
    def __init__ (self, name):
        self.name = name
        self.handle = sim.getObject (name)
        self.update_location ()
        
    def new_location (self, position, orientation):
        sim.setObjectPosition (self.handle, position)
        sim.setObjectOrientation (self.handle, orientation)
        self.update_location ()
        
    def update_location (self):
        self.global_position = sim.getObjectPosition (self.handle)
        self.global_orientation = sim.getObjectOrientation (self.handle)

class CoppeliaRobot (CoppeliaObject):
    
    def __init__ (self, name, radius, wheels):
        # wheel_joints recebe um vetor de objetos do tipo Coppelia_Wheel
        self.radius = radius
        self.wheels = wheels
        CoppeliaObject.__init__ (self, name)

    def calc_linear_velocity (self, position, orientation):
        gain = np.array([[0.5, 0, 0],
                         [0, 0.5, 0],
                         [0, 0, 0.5]])
        delta_pos = np.array (position) - np.array (self.global_position[:2])
        delta_ori = orientation - self.global_orientation[2]
        q_dot = gain @ np.array ((delta_pos[0], delta_pos[1], delta_ori))
        return q_dot

    def inverse_kinematics_omnidirectional (self, velocity, orientation):
        r = self.wheels[0].radius
        L = self.radius
        direct_matrix = np.array([[-r / np.sqrt(3),            0, r / np.sqrt(3)],
                                  [          r / 3, (-2 * r) / 3,          r / 3],
                                  [    r / (3 * L),  r / (3 * L),    r / (3 * L)]])
        inverse_matrix = np.linalg.inv (rotation_z (orientation) @ direct_matrix)
        angular_velocity = inverse_matrix @ velocity
        return angular_velocity

    def follow_path (self, path):
        for stop in path:
            t = 0
            last_time = time.time()
            while np.linalg.norm (self.global_position[:2] - stop) > 0.1 and t < 5:
                angle = np.arctan2 (stop[1], stop[0])
                linear_velocity = self.calc_linear_velocity (stop, angle)
                wheels_velocity = self.inverse_kinematics_omnidirectional (linear_velocity, angle)
                for i in range (3):
                    self.wheels[i].set_velocity (wheels_velocity[i])
                self.update_location ()
                t = t + time.time() - last_time
                last_time = time.time()

class CoppeliaWheel (CoppeliaObject):
    
    def __init__ (self, name, radius):
        self.radius = radius
        CoppeliaObject.__init__ (self, name)

    def set_velocity (self, velocity):
        sim.setJointTargetVelocity (self.handle, velocity)

class CoppeliaLaser (CoppeliaObject):
    
    def __init__ (self, name, laser_range_data, laser_angle_data):
        self.laser_range_data = laser_range_data
        self.laser_angle_data = laser_angle_data

In [ ]:
def is_cell_invalid (cell, world):
    
    if cell [0] < 0: return True
    if cell [1] < 0: return True
    if cell [0] >= len (world[0]): return True
    if cell [1] >= len (world): return True
        
    return False

def reconstruct_path_to (last_node, world):

    path = [last_node.grid_position]
    world[last_node.grid_position[1]][last_node.grid_position[0]] = 3
    while last_node.parent is not None:
        path.append(last_node.parent.grid_position)
        last_node = last_node.parent
        world[last_node.grid_position[1]][last_node.grid_position[0]] = 3

    return path, world

def A_star (start, end, binary_world):

    frontier = []
    path = []
    visited_map = np.zeros ((len(binary_world), len(binary_world[0])))
    binary_world[start[1]][start[0]] = 1
    heapq.heappush (frontier, Node (start, None, 0))
    aux = 0

    while len(frontier) > 0:
        current_node = heapq.heappop(frontier)

        if not (current_node.grid_position - end).any():
            path, binary_world = reconstruct_path_to (current_node, binary_world)
            break

        for shift_one in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
            neighbor_position = (current_node.grid_position + shift_one).astype(int)
            
            if is_cell_invalid (neighbor_position, binary_world): continue
            if binary_world[neighbor_position[1]][neighbor_position[0]] == 0: continue
            if binary_world[neighbor_position[1]][neighbor_position[0]] == 2: continue
            else: binary_world[neighbor_position[1]][neighbor_position[0]] = 2

            neighbor = Node (neighbor_position, current_node, current_node.current_cost + 1)
            neighbor.calc_priority_score (end)
            heapq.heappush (frontier, neighbor)

    
    print_world_grid (binary_world)
        
    return path

![SegmentLocal](imagens/gif_a_star.gif "segment")

In [ ]:
sim.loadScene ("C:/Users/tatia/Desktop/UFMG/RM/TP2/mapas/cave_robotino.ttt")
sim.startSimulation ()

robotino = CoppeliaRobot ("/robotino", 0.135, [])

robotino.wheels.append (CoppeliaWheel ("/wheel0_joint", 0.040))
robotino.wheels.append (CoppeliaWheel ("/wheel1_joint", 0.040))
robotino.wheels.append (CoppeliaWheel ("/wheel2_joint", 0.040))

for i in range (3):
    robotino.wheels[i].set_velocity (0)

goal = CoppeliaObject ("/Goal")

world = World ("C:/Users/tatia/Desktop/UFMG/RM/TP2/mapas/imagens/cave.png", np.array([40,40]))
world_map = world.discretize_world (0.4) # Cell size must be the division of real_dimension by a full number

find_goal (robotino, goal, world_map, 0.4)

sim.stopSimulation ()
time.sleep (5)

In [ ]:
sim.loadScene ("C:/Users/tatia/Desktop/UFMG/RM/TP2/mapas/maze_robotino.ttt")
sim.startSimulation ()

robotino = CoppeliaRobot ("/robotino", 0.135, [])

robotino.wheels.append (CoppeliaWheel ("/wheel0_joint", 0.040))
robotino.wheels.append (CoppeliaWheel ("/wheel1_joint", 0.040))
robotino.wheels.append (CoppeliaWheel ("/wheel2_joint", 0.040))

for i in range (3):
    robotino.wheels[i].set_velocity (0)

goal = CoppeliaObject ("/Goal")

world = World ("C:/Users/tatia/Desktop/UFMG/RM/TP2/mapas/imagens/maze.png", np.array([20,20]))
world_map = world.discretize_world (0.4) # Cell size must be the division of real_dimension by a full number

find_goal (robotino, goal, world_map, 0.4)

sim.stopSimulation ()
time.sleep (5)

#### Funções Campos Potenciais

In [ ]:
def att_force (q, goal, katt = 3.0):
    return katt * (goal - q)

def rep_force(q, obs, R=5, krep=2):
    
    # Obstáculo: (x, y, r)
    v = q[0:2] - obs[0:2]
    d = np.linalg.norm(v)  
    
    rep = (1/d**2) * ((1/d) - (1/R)) * (v/d)    
    
    if d > R:
        rep = np.zeros_like(rep)
    return krep * rep

In [ ]:
def read_sensor_data (range_data_signal_id = "hokuyo_range_data", angle_data_signal_id = "hokuyo_angle_data"):
    
    while sim.getStringSignal (range_data_signal_id) == None:
        continue
        
    string_range_data = sim.getStringSignal (range_data_signal_id)
    string_angle_data = sim.getStringSignal (angle_data_signal_id)

    raw_range_data = sim.unpackFloatTable (string_range_data)
    raw_angle_data = sim.unpackFloatTable (string_angle_data)

    return np.array ([raw_angle_data, raw_range_data]).T

In [ ]:
sim.loadScene ("C:/Users/tatia/Desktop/UFMG/RM/TP2/mapas/cave_pioneer.ttt")
sim.startSimulation ()

robotname = './Pioneer_p3dx'
robotname_R_Motor = './Pioneer_p3dx/Pioneer_p3dx_rightMotor'
robotname_L_Motor = './Pioneer_p3dx/Pioneer_p3dx_leftMotor'
laser_range_data = "hokuyo_range_data"
laser_angle_data = "hokuyo_angle_data"

robotHandle = sim.getObjectHandle(robotname)    

robotLeftMotorHandle  = sim.getObjectHandle(robotname_L_Motor)
robotRightMotorHandle = sim.getObjectHandle(robotname_R_Motor)

robotPos = sim.getObjectPosition(robotHandle, sim.handle_world)
robotOri = sim.getObjectOrientation(robotHandle, sim.handle_world)

goalFrame = sim.getObjectHandle('./Goal')

position_goal = sim.getObjectPosition(goalFrame, sim.handle_world)
orientation_goal = sim.getObjectOrientation(goalFrame, sim.handle_world) 

sim.setObjectPosition(goalFrame, [position_goal[0], position_goal[1], 0], sim.handle_world)
sim.setObjectOrientation(goalFrame, [orientation_goal[0], orientation_goal[1], orientation_goal[2]], sim.handle_world) 

L = 0.381
r = 0.0975
maxv = 1.0
maxw = np.deg2rad(45)

rho = np.inf
     
while rho > .05:

    robotPos = sim.getObjectPosition(robotHandle, sim.handle_world)
    robotOri = sim.getObjectOrientation(robotHandle, sim.handle_world)        
    robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]])        


    laser_data = read_sensor_data(laser_range_data, laser_angle_data)
    distLaser = []

    força_AtracTot = att_force(robotConfig, position_goal)
    print("Força de atração: ", força_AtracTot)

    for i in laser_data:
        f1 = 0
        f2 = 0
        d1 = i[0]  # Correção aqui
        ang1 = i[1]  # Correção aqui
        
        if d1 < 5:
            distLaser.append(d1)
        else:
            continue
    
    dist_repuls = min(distLaser)

    x = dist_repuls * np.cos(ang1)
    y = dist_repuls * np.sin(ang1)
    distLaser = np.array([x ,y, 0])

    força_RepulsTot = rep_force(robotConfig, distLaser, 5)
    print("\n", "força de repulsão: ", força_RepulsTot,"\n")
    dx, dy = força_AtracTot[0:2] + força_RepulsTot[0:2]

    dx1, dy1 = position_goal[:2] - robotConfig[:2]

    rho = np.sqrt(dx1**2 + dy1**2)

    alpha = normalize_angle(-robotOri[2] + np.arctan2(dy,dx))
    beta = normalize_angle(orientation_goal[2] - np.arctan2(dy,dx))
    
    kr = 1
    kt = 1

    print("Força de atração ", dx, " Força de repulsão ", dy, "\n")
    v = kr*(dy*np.cos(robotConfig[2]) + dx*np.sin(robotConfig[2]))
    w = kt*(np.arctan2(dy,dx) - robotConfig[2])
            
    # Limit v,w to +/- max
    
    print("Velocidade do robô ",v," velocidade angular do robot ", w, "\n")
    if v > 0 and v > maxv:
        v = maxv
    elif v < 0 and v < -maxv:
        v = -maxv 
    if w > 0 and w > maxw:
        w = maxw
    elif w < 0 and w < -maxw:
        w = -maxw       
    print("Velocidade do robô 2: ",v," velocidade angular do robot 2: ", w, "\n")

    vr = ((2.0*v) + (w*L))/(2.0*r)
    vl = ((2.0*v) - (w*L))/(2.0*r)
    
    sim.setJointTargetVelocity(robotRightMotorHandle, vr)
    sim.setJointTargetVelocity(robotLeftMotorHandle, vl)

sim.setJointTargetVelocity(robotRightMotorHandle, 0)
sim.setJointTargetVelocity(robotLeftMotorHandle, 0)

sim.stopSimulation ()
time.sleep (5)